# Collect Satellite images of the farmers location from multiple timestamps

In [1]:
from python_scripts.connector import *
from python_scripts.helper import *

from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_ndvi import *
from eval_scripts.evalscript_ndvi_values  import *

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
from sentinelhub.geo_utils import to_wgs84
from sentinelhub.constants import CRS
from sentinelhub import SentinelHubCatalog

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


Succefully connected!


## Step 1: Get the necessary timestamps

In [2]:
def get_time_slots(start, end, chunks):
    """
    Get the avalable time windows (time slots)
    """ 
    datetime_start = datetime.datetime.strptime(start, '%Y-%m-%d')
    datetime_end   = datetime.datetime.strptime(end, '%Y-%m-%d')
    
    tdelta = (datetime_end - datetime_start) / chunks
    edges = [(datetime_start + i*tdelta).date().isoformat() for i in range(chunks)]
    slots = [(edges[i], edges[i+1]) for i in range(len(edges)-1)]
    return slots
    

In [3]:
slots = get_time_slots('2017-01-01', '2022-01-01', 5)
print("Yearly time windows")
for slot in slots:
    print(slot)

Yearly time windows
('2017-01-01', '2018-01-01')
('2018-01-01', '2019-01-01')
('2019-01-01', '2020-01-01')
('2020-01-01', '2020-12-31')


## Step 2: Get the necessary coordinates 

In [4]:
# convert coordinations_file (csv file) into a dataframe
coordinations_df = pd.read_excel('data/locations/FarmerLocationExtract4Interns_sentinel-hub.xlsx')

In [5]:
# Based on how far wail is with the conversion of the coordinates of the farmers location, we need to check whether or 
# the coordination needs to mdofied. 
coordinations_df

,Unnamed: 0,Name,Mobile (anonymized),Mouza,GP,Block,Land_ID1,Land_Size2,P1_Latitude,P1_Longitude
0,0,TEMP_NAME,PHONE01,Baksa,11 No Arrah,Nayagram,40,0.167,21.5439876,87.118996
1,1,TEMP_NAME,PHONE02,Banskhali,11 No Arrah,Nayagram,100/53,0.333,21.535550432,87.454971
2,2,TEMP_NAME,PHONE03,Pathrasol,12 No Chandrarekha,Nayagram,51,0.333,21.581197383,87.157488
3,3,TEMP_NAME,PHONE04,Ramkrishnapur,11 No Arrah,Nayagram,36/947,0.167,21.535747928,87.139477
4,4,TEMP_NAME,PHONE05,Bamdiha,11 No Arrah,Nayagram,92,0.333,21.55262056,87.229918
...,...,...,...,...,...,...,...,...,...,...
244,245,TEMP_NAME,PHONE246,Bamdiha,11 No Arrah,Nayagram,230,0.260,21.5547784,87.216591
245,246,TEMP_NAME,PHONE247,Bamdiha,11 No Arrah,Nayagram,47/1,0.260,21.55475356,87.216591
246,247,TEMP_NAME,PHONE248,Bamdiha,11 No Arrah,Nayagram,285,0.520,21.52133968,87.530772
247,248,TEMP_NAME,PHONE249,Bamdiha,11 No Arrah,Nayagram,51,0.520,21.55487128,87.215806


In [6]:
coordinations_df.columns

Index(['Unnamed: 0', 'Name', 'Mobile (anonymized)', 'Mouza', 'GP', 'Block',
       'Land_ID1', 'Land_Size2', 'P1_Latitude', 'P1_Longitude'],
      dtype='object')

In [7]:
def bbox_converter(x):
    """
    Convert coordinates (longitude, latitude) in to a bbox in WGS84 format.
    
    return [long, lat, long, lat]
    """
    lng, lat = to_wgs84(x['P1_Longitude'], x['P1_Latitude'], CRS.WGS84)
    coords = [lng - 0.1, lat - 0.1, lng + 0.1, lat + 0.1]
    return coords

In [8]:
geolocations  = coordinations_df[['P1_Longitude', 'P1_Latitude']].copy()

In [9]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

,P1_Longitude,P1_Latitude
14,87.242824,"21.54""95616"
15,87.234883,"21.54""8830"
16,87.234738,"21.54""329472"


In [10]:
geolocations['P1_Latitude'] = geolocations['P1_Latitude'].str.replace('"', '')

In [11]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

,P1_Longitude,P1_Latitude


In [12]:
geolocations = geolocations.astype("float64")

In [13]:
geolocations.dtypes

P1_Longitude    float64
P1_Latitude     float64
dtype: object

In [14]:
geolocations

,P1_Longitude,P1_Latitude
0,87.118996,21.543988
1,87.454971,21.535550
2,87.157488,21.581197
3,87.139477,21.535748
4,87.229918,21.552621
...,...,...
244,87.216591,21.554778
245,87.216591,21.554754
246,87.530772,21.521340
247,87.215806,21.554871


## Create requests for the labels

In [15]:
def map_ndvi_label(ndvi):
    """
    Categorizing range of ndvi values to a label.
    We have two categories: 
    1. trees - 1
    2. not trees - 0
    We categorize evey ndvi values that is higher than our threshold(0.6) to be trees (1).
    Everything that is equal or less than our threshold (0.6) we be 'not trees' (0): 
    """
    min_ndvi = 0.6
    ndvi_copy = ndvi.copy()

    labels = np.where(ndvi_copy > min_ndvi, 1, 0)
     
    return labels

In [16]:
def labels_request(coords, slot, config):
    """
    Download labels in binary array from sentinel hub.
    
    return: numpy array of 0 and 1
    """
    ndvi_values = sentinel_request(evalscript_ndvi_values, coords, slot, config, False, '../data/ndvi_values_images',other_args=least_clouds())
    ndvi_labels= map_ndvi_label(ndvi_values)
    
    return ndvi_labels
    

## Download all the satellite images 

In [17]:
import time # time the performance

In [18]:
def least_clouds():
    return { 
    "dataFilter": { 
        "maxCloudCoverage": 10
        } 
    } 

In [20]:
for index, row in geolocations.iterrows(): # iterate through each location
    
    coords = bbox_converter(row) # convert each location to a bbox
    
    
    labels = [] # TODO check if this is the correct way to store the labels 
    
    start = time.perf_counter()
    for slot in slots:
        sentinel_request(evalscript_true_color, coords,slot, config, True,'../data/images', other_args=least_clouds())
        sentinel_request(evalscript_ndvi, coords,slot, config, True,'../data/mask', other_args=least_clouds())
        request_geojson(evalscript_true_color, coords,slot, config, True,'../data/geojson', other_args=least_clouds())
        
        # get the labels
        binary_arr_labels = labels_request(coords, slot, config)
        labels.append(binary_arr_labels) # TODO  check if this is correct and efficient
    end = time.perf_counter()
    print("Done in: {} seconds".format( end - start))

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"error":{"status":400,"reason":"Bad Request","message":"Format application/json does not support sample type AUTO.","code":"RENDERER_EXCEPTION"}}"

Below you wil find a function the plot images for one specific scene/area and the given timestamps

In [ ]:
def plot_images(data, slots):
    # some stuff for pretty plots
    ncols = 2
    nrows = 2
    aspect_ratio = betsiboka_size[0] / betsiboka_size[1]
    subplot_kw = {'xticks': [], 'yticks': [], 'frame_on': False}

    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows),
                            subplot_kw=subplot_kw)

    for idx, image in enumerate(data):
        ax = axs[idx // ncols][idx % ncols]
        ax.imshow(np.clip(image * 2.5/255, 0, 1))
        ax.set_title(f'{slots[idx][0]}  -  {slots[idx][1]}', fontsize=10)

    plt.tight_layout()